# Image Bootstrap 

To increase the number of training images, we can take the originals and 
- convert to black and white
- rotate
- flip horizontally


## Imports

In [1]:
from PIL import Image, ImageEnhance
import os
import numpy as np

## Global configuration settings

In [2]:
#image_dir = '../images/test'
#image_dir = '../images/bootstrap_truck'
#image_dir = '../images/truck/test/depth_0'
image_dir = '../images/truck/validate/depth_0'

aug_separated = False  # True to put new images in a new folder under image_dir/augmented
                      # False to keep in same folder as original

## Image list

In [3]:
# Optional peek at the list of input images
!ls {image_dir}

2020-ram-1500-dt-whybuy-towing-boat-lake-wood_9c49b99c885cf843e2e947809120b2c1-839x310.jpg
8f87e2dd99bc5e37a5ff1a6491cc9dbc.jpg
black-ram-truck-diamondback-hd-rain-1920.jpg


## Transformation functions

In [4]:
# TODO: if use of the augmented dir was selected, check for existence and create if needed

In [5]:
# Remove .DS_store from the list of files
def rm_DS(mlist): 
    if ('.DS_Store' in mlist): mlist.remove('.DS_Store')
    return mlist

In [6]:
# Remove other items from the list of files in a list
def rm_config(mlist,item): 
    if (item in mlist): mlist.remove(item)
    return mlist

In [7]:
def make_hflip_version(path,fname,augdir = False):
    if (augdir):
        finish_sep = '/augmented/'
    else:
        finish_sep = '/'
                
    new_fname = 'hflip_' + fname
    fullpath_start  = path + '/' + fname
    fullpath_finish = path + finish_sep + new_fname
    
    start_img = Image.open(fullpath_start)
    converted = start_img.transpose(method=Image.FLIP_LEFT_RIGHT)
    converted.save(fullpath_finish)
    start_img.close()
    
    return new_fname

In [8]:
def make_bw_version(path,fname,augdir = False, debug=False):
    if (augdir):
        finish_sep = '/augmented/'
    else:
        finish_sep = '/'
        
    if (debug):
        print("DEBUG: make_bw_version:")
        print("DEBUG: make_bw_version:")
        print("DEBUG: make_bw_version:")
        
    new_fname = 'bw_' + fname
    fullpath_start  = path + '/' + fname
    fullpath_finish = path + finish_sep + new_fname
    
    start_img = Image.open(fullpath_start)
    converted = start_img.convert(mode='L')
    converted .save(fullpath_finish)
    start_img.close()
    
    return new_fname

In [9]:
def make_rot_version(path,fname,rotation,augdir = False):
    if (augdir):
        finish_sep = '/augmented/'
    else:
        finish_sep = '/'
            
    new_fname = 'rot-'+ str(rotation) + '_' + fname
    fullpath_start  = path + '/' + fname
    fullpath_finish = path + finish_sep + new_fname
    
    start_img = Image.open(fullpath_start)
    converted = start_img.rotate(rotation)
    converted.save(fullpath_finish)
    start_img.close()
    
    return new_fname

In [10]:
def make_enhanced_version(path,fname,factor,augdir = False):
    '''factor of 1 means no change
                <1 means less color (brightness, contrast, etc.)
                >1 means actual enhancement'''
    if (augdir):
        finish_sep = '/augmented/'
    else:
        finish_sep = '/'
                
    new_fname = 'enh-'+ str(factor) + '_' + fname
    fullpath_start  = path + '/' + fname
    fullpath_finish = path + finish_sep + new_fname
    
    start_img = Image.open(fullpath_start)
    converted = ImageEnhance.Sharpness(start_img).enhance(factor)
    converted.save(fullpath_finish)
    start_img.close()
    
    return new_fname

In [11]:
def make_contrast_version(path,fname,factor,augdir = False):
    if (augdir):
        finish_sep = '/augmented/'
    else:
        finish_sep = '/'
        
    new_fname = 'con-'+ str(factor) + '_' + fname
    fullpath_start  = path + '/' + fname
    fullpath_finish = path + finish_sep + new_fname
    
    start_img = Image.open(fullpath_start)
    converted = ImageEnhance.Contrast(start_img).enhance(factor)
    converted.save(fullpath_finish)
    start_img.close()
    
    return new_fname

In [12]:
def make_crop_version(path,fname,factor,augdir = False):
    if (augdir):
        finish_sep = '/augmented/'
    else:
        finish_sep = '/'
        
    new_fname = 'crop-'+ str(factor) + '_' + fname
    fullpath_start  = path + '/' + fname
    fullpath_finish = path + finish_sep + new_fname
    
    start_img = Image.open(fullpath_start)
    wid, ht = start_img.size
    (left, upper, right, lower) = (0, 0, wid, ht*factor)
    
    converted = start_img.crop((left, upper, right, lower))
    converted.save(fullpath_finish)
    start_img.close()
    
    return new_fname

## Make Transforms

In [13]:
print(image_dir)

../images/truck/test/depth_0


In [14]:
# get files in dir into a list; remove '.DS_Store'
image_list = rm_config(rm_DS(os.listdir(image_dir)),'augmented')

In [15]:
# optional check of the image_list
image_list

['black-ram-truck-diamondback-hd-rain-1920.jpg',
 '2020-ram-1500-dt-whybuy-towing-boat-lake-wood_9c49b99c885cf843e2e947809120b2c1-839x310.jpg',
 '8f87e2dd99bc5e37a5ff1a6491cc9dbc.jpg']

In [16]:
# run transforms on the files in the list
for image in image_list: 

    # Include BW versions to avoid training on color variation
    make_bw_version(image_dir, image, augdir=aug_separated)
    
    # Simple horizontal flip so that we don't inadvertently
    #    train on left-facing are in shallow, right-facing are in deep, or similar
    make_hflip_version(image_dir, image, augdir=aug_separated)

    # Rotation of up to 5 degrees may account for handheld horizon variation in photos
    cx = round(np.random.uniform(1,5),1)     
    make_rot_version(image_dir, image, cx, augdir=aug_separated)

    # Crops that retained less than 0.8 of the image were too extreme
    cx = round(np.random.uniform(.8,.9),2)
    make_crop_version(image_dir, image, cx, augdir=aug_separated)

### Experiments

In [ ]:
# TODO: Remove this entire section once we have the transforms we want

In [17]:
# path = '../images/test'
# fname = '34384058_05039deebe_z.jpg'
# # 
# # Do selected transforms
# bw_name = make_bw_version(path,fname)

In [18]:
#make_rot_version(path,bw_name,5)

In [19]:
#make_enhanced_version(path, fname, 0.01)

In [20]:
#con = make_contrast_version(path, fname, 0.3)
#con

In [21]:
#crop = make_crop_version(path, con, 0.7)
#crop

In [22]:
#c4 = make_crop_version(path, fname, 0.4)
#c4

In [23]:
#round(np.random.uniform(.6,.9),2)

In [24]:
# # get image size

# fullpath_start = path + '/' + fname
# fullpath_finish = path + '/' + 'crop-' + fname

In [25]:
#start_img = Image.open(fullpath_start)

In [26]:
#wid, ht = start_img.size

In [27]:
#(left, upper, right, lower) = (0, 0, wid, ht/2)

In [28]:
#converted = start_img.crop((left, upper, right, lower))

In [29]:

#converted.save(fullpath_finish)

In [30]:
#start_img.close()